In [1]:
from torch.utils.data import DataLoader, WeightedRandomSampler
from resnet import ResNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, RocCurveDisplay
import numpy as np
from custom_dataset import FinalDataset
import torch
from torch import nn
import torch.optim as optim
import matplotlib.pyplot as plt
from FocalLoss import FocalLoss

In [2]:
file_path= "./data/Dataset.hdf5"
dataset = FinalDataset(file_path ,downsample=True,majority_ratio=0.50)

In [4]:
# labels = dataset.get_labels()

In [5]:
# unique_classes, class_counts = np.unique(labels, return_counts=True)
# class_counts_dict = dict(zip(unique_classes, class_counts))
# class_counts_dict

In [6]:
# complete_dataset = FinalDataset(file_path)

In [7]:
# len(complete_dataset)

In [10]:
print(len(dataset))

16380


In [3]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset.indices)
    model.train()
    total_loss = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y).mean()
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        if batch % 100 == 0:
            print(f"loss: {loss.item():>7f} [{batch * len(X):>5d}/{size:>5d}]")
    return total_loss / len(dataloader)


def test_loop(dataloader, model, loss_fn, device,val = False):
    size = len(dataloader.dataset.indices)
    total_loss = 0
    model.eval()
    all_preds, all_targets = [], []
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y).mean()
            total_loss += loss.item()
            all_preds.append(pred.cpu())
            all_targets.append(y.cpu())
    
    all_preds = torch.cat(all_preds)
    all_targets = torch.cat(all_targets)

    probabilities = torch.sigmoid(all_preds)

    targets = all_targets.int().numpy()

    for threshold in np.arange(0.1, 1.0, 0.1):
        print(f"\n==== Threshold: {threshold:.1f} ====")
        predicted = (probabilities > threshold).int().numpy()

        cm = confusion_matrix(targets, predicted)
        print("Confusion Matrix:")
        print(cm)
        
        report = classification_report(targets, predicted, digits=4, zero_division=0)
        print("Classification Report:")
        print(report)
    if val:
        RocCurveDisplay.from_predictions(all_targets, probabilities, name="ROC Curve")
        plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
        plt.title('ROC Curve')
        plt.savefig('roc_curve_SDG_50_BIGGER.png')
        plt.show()
    return total_loss / len(dataloader)



Parametri

In [ ]:
# Focal loss parameters
gamma = 2.0
alpha = 0.95

# ResNet parameters
in_channels = 12
out_channels = 64

# Optimizer parameters
learning_rate = 1e-4
momentum = 0.9
weight_decay = 1e-4

# K-fold cross-validation parameters
random_state = 42

# Training parameters
epochs = 100
batch_size = 32

patience, count = 5,0

In [5]:
train_val_idx, test_idx = train_test_split(
    np.arange(len(dataset)),
    test_size=0.15,
    random_state=random_state,
    stratify=dataset.get_labels()
)
train_val_idx.sort()
train_idx, val_idx = train_test_split(
    train_val_idx,
    test_size=0.17647,  # ~15% of total
    random_state=random_state,
    stratify=dataset.labels[train_val_idx]
)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset labels
# labels = np.array(dataset.get_labels() ,dtype=np.int32)
# labels = labels.squeeze()

# Class counts and pos_weight for BCEWithLogitsLoss
# unique_classes, class_counts = np.unique(labels, return_counts=True)
# class_counts_dict = dict(zip(unique_classes, class_counts))
# print(f"Class counts: {class_counts_dict}")

# pos_weight = torch.tensor(155609/ 8190).to(device)

# # with 95% majority ratio
# loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

# with 50% majority ratio
loss_fn = nn.BCEWithLogitsLoss()

# with Focal Loss and 95% majority ratio
# loss_fn = FocalLoss(gamma, alpha)

# Model and optimizer
model = ResNet(in_channels, out_channels).to(device)
# optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, weight_decay=1e-4)

train_loader = DataLoader(FinalDataset("./data/Dataset.hdf5", indices = dataset.indices[train_idx]), batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(FinalDataset("./data/Dataset.hdf5", indices = dataset.indices[val_idx]),   batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(FinalDataset("./data/Dataset.hdf5", indices = dataset.indices[test_idx]),  batch_size=batch_size, shuffle=False)

train_losses = []
val_losses = []
best_loss = float('inf')


# Single train/val loop
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loss = train_loop(train_loader, model, loss_fn, optimizer, device)
    val_loss   = test_loop(val_loader, model, loss_fn, device)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    if val_loss < best_loss:
        best_loss = val_loss
        count = 0
    else:
        count += 1

    if count > patience:
        print("Early stopping")
        break

    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

# Test after training
test_loss = test_loop(test_loader, model, loss_fn, device, val=True)
print(f"Final Test Loss: {test_loss:.4f}")
fig, ax1 = plt.subplots()

ax1.plot(range(1, len(train_losses) + 1), train_losses, label="Train Loss", color='tab:blue')
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Train Loss", color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')

ax2 = ax1.twinx()  
ax2.plot(range(1, len(val_losses) + 1), val_losses, label="Test Loss", color='tab:red')
ax2.set_ylabel("Test Loss", color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')

ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.title(f"Train and Test Loss")
plt.tight_layout()
plt.savefig(f"train_test_loss_SDG_50_BIGGER.png")
plt.show()

torch.save(model.state_dict(), f"model_SDG_50_BIGGER.pth")
print(f"Saved model")

print("Training Completed!")


Epoch 1
-------------------------------
loss: 0.683765 [    0/11466]

==== Threshold: 0.1 ====
Confusion Matrix:
[[   0 1188]
 [   0 1269]]
Classification Report:
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      1188
           1     0.5165    1.0000    0.6812      1269

    accuracy                         0.5165      2457
   macro avg     0.2582    0.5000    0.3406      2457
weighted avg     0.2668    0.5165    0.3518      2457


==== Threshold: 0.2 ====
Confusion Matrix:
[[   0 1188]
 [   0 1269]]
Classification Report:
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      1188
           1     0.5165    1.0000    0.6812      1269

    accuracy                         0.5165      2457
   macro avg     0.2582    0.5000    0.3406      2457
weighted avg     0.2668    0.5165    0.3518      2457


==== Threshold: 0.3 ====
Confusion Matrix:
[[   1 1187]
 [   0 1269]]
Classification Rep

KeyboardInterrupt: 